In [14]:
dimport cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
CELL_SIZE = 20
NCLASSES = 10
TRAIN_RATIO = 0.8

In [3]:
digits_img = cv2.imread('../data/digits.png', 0)
digits = [np.hsplit(r, digits_img.shape[1] // CELL_SIZE) 
          for r in np.vsplit(digits_img, digits_img.shape[0] // CELL_SIZE)]
digits = np.array(digits).reshape(-1, CELL_SIZE, CELL_SIZE)
nsamples = digits.shape[0]
labels = np.repeat(np.arange(NCLASSES), nsamples // NCLASSES)

In [4]:
for i in range(nsamples):
    m = cv2.moments(digits[i])
    if m['mu02'] > 1e-3:
        s = m['mu11'] / m['mu02']
        M = np.float32([[1, -s, 0.5*CELL_SIZE*s], 
                        [0, 1, 0]])
        digits[i] = cv2.warpAffine(digits[i], M, (CELL_SIZE, CELL_SIZE))

In [5]:
perm = np.random.permutation(nsamples)
digits = digits[perm]
labels = labels[perm]

In [6]:
ntrain = int(TRAIN_RATIO * nsamples)
ntest = nsamples - ntrain

- win_size : 탐지 윈도우 사이즈

- block_size : 픽셀에서의 블록 사이즈 , 현재 (16,16) 만 지원됨

- block_stride : 블록의 보폭, 셀 크기의 배수여야 함  

- cell_size : 셀 사이즈

- nbins : bin의 갯수, 현재 9 bin 만 지원됨



출처: https://eehoeskrap.tistory.com/122 

In [7]:
def calc_hog(digits):
    win_size = (20, 20)
    block_size = (10, 10)
    block_stride = (10, 10)
    cell_size = (10, 10)
    nbins = 9
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    samples = []
    for d in digits: samples.append(hog.compute(d))
    return np.array(samples, np.float32)

In [8]:
fea_hog_train = calc_hog(digits[:ntrain])
fea_hog_test = calc_hog(digits[ntrain:])
labels_train, labels_test = labels[:ntrain], labels[ntrain:]

In [9]:
K = 3
knn_model = cv2.ml.KNearest_create()
knn_model.train(fea_hog_train, cv2.ml.ROW_SAMPLE, labels_train)

True

In [10]:
svm_model = cv2.ml.SVM_create()
svm_model.setGamma(2)
svm_model.setC(1)
svm_model.setKernel(cv2.ml.SVM_RBF)
svm_model.setType(cv2.ml.SVM_C_SVC)
svm_model.train(fea_hog_train, cv2.ml.ROW_SAMPLE, labels_train)

True

In [11]:
def eval_model(fea, labels, fpred):
    pred = fpred(fea).astype(np.int32)
    acc = (pred.T == labels).mean()*100
    
    conf_mat = np.zeros((NCLASSES, NCLASSES), np.int32)
    for c_gt, c_pred in zip(labels, pred):
        conf_mat[c_gt, c_pred] += 1
    
    return acc, conf_mat

In [12]:
knn_acc, knn_conf_mat = eval_model(fea_hog_test, labels_test, lambda fea: knn_model.findNearest(fea, K)[1])
print('KNN accuracy (%):', knn_acc)
print('KNN confusion matrix:')
print(knn_conf_mat)

KNN accuracy (%): 91.8
KNN confusion matrix:
[[100   0   0   0   0   0   1   0   0   0]
 [  0  89   2   0   1   0   0   0   0   0]
 [  0   0  96   3   0   0   0   0   1   0]
 [  1   0   1  99   1   3   0   1   2   0]
 [  2   0   3   6  90   0   5   1   1   6]
 [  0   0   0   2   1  93   1   1   3   2]
 [  1   0   1   0   0   0 102   0   1   0]
 [  0   2   2   5   0   1   0  72   0   3]
 [  1   0   0   2   1   2   0   0 101   0]
 [  1   0   2   2   2   0   0   0   2  76]]


In [13]:
svm_acc, svm_conf_mat = eval_model(fea_hog_test, labels_test, lambda fea: svm_model.predict(fea)[1])
print('SVM accuracy (%):', svm_acc)
print('SVM confusion matrix:')
print(svm_conf_mat)

SVM accuracy (%): 94.6
SVM confusion matrix:
[[ 98   0   0   0   0   0   2   0   0   1]
 [  0  90   0   0   2   0   0   0   0   0]
 [  0   0  94   2   2   0   0   2   0   0]
 [  0   0   2 103   0   2   0   1   0   0]
 [  1   0   1   1 103   0   3   2   0   3]
 [  0   0   0   1   0  99   0   2   1   0]
 [  0   0   1   0   0   0 103   0   1   0]
 [  0   1   2   0   0   2   0  77   0   3]
 [  1   0   0   1   2   2   0   0 101   0]
 [  0   0   0   1   3   0   0   0   3  78]]
